In [1]:
import pandas as pd
import math
from dateutil.relativedelta import relativedelta
from datetime import datetime
import re
import copy
from sqlalchemy import create_engine
import calendar

In [2]:
conn_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(user='postgres',
                                                                     pwd='123',
                                                                     host='localhost',
                                                                     port='5432',
                                                                     db='postgres'
                                                                     )
engine = create_engine(conn_string, paramstyle='format', isolation_level="READ UNCOMMITTED")

In [3]:
path = 'D:/NE (No AS n SK)/'
state = 'TR'

In [4]:
slab_fields = pd.read_excel(r'C:\Users\CDMA\Downloads\slab_fields_prep_v2.xlsx',sheet_name = 'slab_fields_prep')

In [5]:
slab_new = pd.read_csv(r'C:\Users\CDMA\Downloads\vm_tax_slab_new_transformed.csv')
slab_new = slab_new[slab_new['state_cd']==state]
slab_new = slab_new[slab_new['vh_class'].notnull()]

In [6]:
slab_new['date_from'] = slab_new['date_from'].astype('datetime64[ns]')
slab_new['date_to'] = slab_new['date_to'].astype('datetime64[ns]')
slab_new.reset_index(drop=True, inplace=True)

In [7]:
slab_add_new = pd.read_csv(r'D:\Jharkhand python\vm_tax_slab_add_new.csv')
slab_add_new = slab_add_new[slab_add_new['state_cd']==state]

In [8]:
vt_tax_based_on = pd.read_csv(path+'/vt_tax_based_on.csv')
#vt_tax_based_on_permit_info = pd.read_csv(r'D:\AS\vt_tax_based_on_permit_info.csv')
#vt_tax_based_on = pd.read_sql_query('Select * from vt_tax_based_on where state_cd ='+"'{}'".format(state),con=engine2)
vt_tax_based_on = vt_tax_based_on[vt_tax_based_on['state_cd']==state]

In [9]:
vm_vh_class=pd.read_csv(r'D:\Jharkhand python\vm_vh_class.csv')

In [10]:
vt_owner = pd.read_csv(path+'/vt_owner.csv')
vt_owner = vt_owner[vt_owner['state_cd']==state]
#vt_owner = pd.read_sql_query('Select * from vt_owner where state_cd ='+"'{}'".format(state),con=engine2)

C:\Users\CDMA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (20,59) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
tm_office = pd.read_csv('D:/RJ/tm_office.csv')
#tm_office.head()
tm_office = tm_office[tm_office['state_cd']==state]

In [12]:
#Turn into first letter capital in each word
def change_col_names(cols):
    pp=[]
    for j in cols :
        p= j.replace('_',' ').title()
        pp.append(p)
        #df.rename(columns={j: p},inplace=True)
    return pp

In [13]:
def calculate_formula(formula_df,params_info):
    #Calculate the Formula one by one
    ind=[]
    cff_formula = []
    cc_formula=[]
    ccf_result=[]
    cr_rate=[]
    #c_add_code=[]
    #reb_df = formula_df[formula_df['type_flag']=='R']
    for l in range(len(formula_df)):
        #print('l',l)
        condition_formula = formula_df['condition_formula'].to_list()[l]
        if '\r' in condition_formula:
            condition_formula = condition_formula.replace('\r','')
        if '\n' in condition_formula:
            condition_formula = condition_formula.replace('\n','')

        cf_params = []
        for k in re.split(r'[()\*\s]\s*',condition_formula):
            if '<' in k and '>' in k:
                cf_params.append(k[k.find('<'):(k.find('>')+1)])
        cf_params = list(set(cf_params))
        c_formula = condition_formula
        #print(c_formula)
        c_formula = condition_form_convert(cf_params,params_info, c_formula)
        #print(c_formula)
        for m in cf_params:
            if m != '<>':
                val = str(params_info[params_info['code']==m]['P_value'].to_list()[0])                
                c_formula = c_formula.replace(str(m),val)
                #print(c_formula,str(m),val)
        c_formula = c_formula.replace(repr('nan'),'-100')
        #print(c_formula)
        if c_formula.lower() != 'false' and formula_df['type_flag'].to_list()[l] != 'R' and 'nan' not in c_formula.lower():
            cf_result = pd.read_sql_query(repr('select '+c_formula)[1:-1],con=engine)
            if 'case' not in cf_result.columns:
                cf_result = cf_result.loc[0,'?column?']
            else:
                cf_result = cf_result.loc[0,'case']
        else:
            cf_result = False
        if cf_result==True:
            #print('Ind',ind,l)
            ind.append(l)
        cff_formula.append(condition_formula)
        cc_formula.append(c_formula)
        ccf_result.append(cf_result)
        cr_rate.append(formula_df.loc[:,'rate_formula'].to_list()[l])
        
    formula_df.reset_index(drop=True, inplace=True)
    summary_df = pd.DataFrame(list(zip(cff_formula,cc_formula, ccf_result,cr_rate)), columns =['Condition Formula', 'Condition Result','Result','Rate Formula']) 
    return [ind,summary_df]

In [14]:
def condition_form_convert(cf_params,params_info, c_formula):
    for m in cf_params:
        if m != '<>':
            val = str(params_info[params_info['code']==m]['P_value'].to_list()[0])
            # Change '' to \'\' in case of string val
            if val.isdigit()==False:
                if '.' in val:
                    v = val.replace('.','')
                    if '-' in v:
                        v = v.replace('-','')
                    if v.isdigit():
                        val=val
                    else:
                        val=repr(val)
                else:
                    val=repr(val)
            else:
                val=val
            c_formula = c_formula.replace(str(m),val)
    return c_formula
'''
        else:
            c_formula = c_formula.replace('<>','!=')

    # Change '=' to '=='
    if '<=' in c_formula:
        c_formula=c_formula.replace('<=','@')
    if '>=' in c_formula:
        c_formula=c_formula.replace('>=',';')
    if '!=' in c_formula:
        c_formula=c_formula.replace('!=','?')
    if '=' in c_formula:
        c_formula = c_formula.replace('=','==')
    if '@' in c_formula:
        c_formula=c_formula.replace('@','<=')
    if ';' in c_formula:
        c_formula=c_formula.replace(';','>=')
    if '?' in c_formula:
        c_formula = c_formula.replace('?','!=')
    return c_formula
'''

"\n        else:\n            c_formula = c_formula.replace('<>','!=')\n\n    # Change '=' to '=='\n    if '<=' in c_formula:\n        c_formula=c_formula.replace('<=','@')\n    if '>=' in c_formula:\n        c_formula=c_formula.replace('>=',';')\n    if '!=' in c_formula:\n        c_formula=c_formula.replace('!=','?')\n    if '=' in c_formula:\n        c_formula = c_formula.replace('=','==')\n    if '@' in c_formula:\n        c_formula=c_formula.replace('@','<=')\n    if ';' in c_formula:\n        c_formula=c_formula.replace(';','>=')\n    if '?' in c_formula:\n        c_formula = c_formula.replace('?','!=')\n    return c_formula\n"

In [15]:
def calculate_rates(true_df,params_info):
    t_params = []
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for k in re.split(r'[()\*\s]\s*',r_formula):
            if '<' in k and '>' in k:
                t_params.append(k[k.find('<'):(k.find('>')+1)])

    t_params = list(set(t_params))
    #print('t_params',t_params)
    rates=[]
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for m in t_params:
            val = str(params_info[params_info['code']==m]['P_value'].to_list()[0])
            # Change '' to \'\' in case of string val
            if val.isdigit()==False:
                if '.' in val:
                    v = val.replace('.','')
                    if '-' in v:
                        v = v.replace('-','')
                    if v.isdigit():
                        val=val
                    else:
                        val=repr(val)
                else:
                    val=repr(val)
            else:
                val=val
            r_formula = r_formula.replace(str(m),val)
        
        #print('r_formula',r_formula)
            
        #rate = eval(r_formula.lower())
        rate = pd.read_sql_query(repr('select '+r_formula)[1:-1],con=engine)
        #print(rate)
        if 'case' not in rate.columns:
            rate = rate.loc[0,'?column?']
        else:
            rate = rate.loc[0,'case']
        #rate = rate.loc[0,'?column?']
        #print(rate)
        rates.append(rate)        
    true_df['Rate']=rates
    return true_df

In [16]:
def tbo_last_rcpt(regn_no,before):
    tbo_df = vt_tax_based_on.loc[vt_tax_based_on['regn_no']==regn_no]
    tbo_df= copy.deepcopy(tbo_df)
    #print('TBO shape',tbo_df.shape)
    #print(tbo_df.shape)
    #tbo_df = tbo_df.reset_index(drop=True, inplace=True)
    tbo_df['op_dt'] = tbo_df.loc[:,'op_dt'].apply(lambda x: datetime.strptime(x[:10], '%d-%m-%Y'))
    tbo_df = tbo_df.sort_values(by='op_dt', ascending=False)
    #tbo_df = tbo_df.reset_index(drop=True, inplace=True)
    if len(tbo_df)>0:#print(tbo_df.shape)
        for n in tbo_df.index:
            x = before-tbo_df.loc[n,'op_dt']
            #x = x.years*365 + x.months*30 + x.days
            if x.days>=0:
                last_rcpt = tbo_df.loc[n,'rcpt_no']
                break
            last_rcpt=''
        if last_rcpt=='':
            print('No prior receipt info!!! Rcpt:',' Regn No:',regn_no)
            return 0
        else:
            return last_rcpt
    else:
        return 0

In [17]:
def owner_or_tbo(regn_no,params_info,param_info_owner,ot_df,tab,new_vch,rcpt_dt,tax_mode):
    for pi in range(len(param_info_owner)):
        col = param_info_owner['column_name'].to_list()[pi][1:-2]
        par = param_info_owner['column - slab_field'].to_list()[pi]
        code = param_info_owner['code'].to_list()[pi]
        #col= col.replace('_',' ').title()
        p_value = ''
        #print(par,code,tab)
        #if tab=='own':
        #    p_date = 'Purchase Dt'
        #else:
        p_date= 'purchase_dt'
        purchase_dt = ot_df[p_date].to_list()[0]
        
        p_valid_date = purchase_dt[:2].isdigit()
        if p_valid_date==False and tab=='tbo':
            #print(p_valid_date,tab)
            purchase_dt = vt_owner.loc[vt_owner['regn_no']==regn_no][p_date].to_list()[0]
        p_valid_date = purchase_dt[:2].isdigit()
        #print(purchase_dt)
        if purchase_dt[:4].isdigit() and p_valid_date==True:
            purchase_dt =datetime.strptime(purchase_dt,'%Y-%m-%d')
        elif purchase_dt[3:6].isalpha() and p_valid_date==True:
            purchase_dt =datetime.strptime(purchase_dt,'%d-%b-%Y')
        elif p_valid_date==True:
            purchase_dt =datetime.strptime(purchase_dt,'%d-%m-%Y')
        if par=='vch_age' and p_valid_date==True:
            p_value = relativedelta(rcpt_dt, purchase_dt)
            #print('p_value',p_value)
            p_value = p_value.years
        elif par=='vch_age_months'and p_valid_date==True:
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = (p_value.years *12) + p_value.months
        elif par=='purchase_dt_vch_age'and p_valid_date==True:
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = p_value.years
        elif par=='fin_vch_age'and p_valid_date==True:
            #print(datetime.strptime(ot_df['purchase_dt'].to_list()[0],'%d-%m-%Y'))
            p_value = relativedelta(rcpt_dt, purchase_dt)
            p_value = p_value.years
        elif par=='tax_month_duration':
            p_value = relativedelta(tax_upto, tax_from)
            p_value = p_value.years *12 + p_value.months
        elif par=='payment_date':
            p_value = rcpt_dt
        #elif par=='distance_run_in_month':
        #    p_value = vt_tax_based_on['distance_run_in_quarter']/4
        #elif par=='transaction_pur_cd':
        #    appl_no = vp_appl_rcpt_mapping[vp_appl_rcpt_mapping['rcpt_no']==rcpt_no]['appl_no'].to_list()[0]
        #    if len(va_details[va_details['appl_no']==appl_no][col].to_list())==0:
        #        print(i,'No pur_cd available in va_details')
        #        print('Appl no',appl_no,' !!! Rcpt:',rcpt_no,' Regn No:',regn_no,'vh_class',vh_class)
        #        continue
        #    else:
        #        p_value = va_details[va_details['appl_no']==appl_no][col].to_list()[0]
        #print(p_value)
        elif par=='vch_type':
            p_value = vm_vh_class[vm_vh_class['Vh Class']==vh_class]['Class Type'].to_list()[0]
        elif par=='new_vch':
            p_value = new_vch
        elif par=='tax_mode':
            p_value = tax_mode
        elif par=='permit_sub_catg':
            if 'pmt_catg' in ot_df.columns:
                p_value = ot_df['pmt_catg'].to_list()[0]
            else:
                p_value = float("nan")
        elif par=='delay_days':
            #print(rcpt_dt[:10],' ',tax_from)
            p_value = rcpt_dt-tax_from
            #p_value = p_value.years * 365 + p_value.months * 30 + p_value.days
            if p_value.days > 0:
                p_value = p_value.days
            elif p_value == 0:
                p_value = 1
            else:
                p_value = 0
            #print('delayed',p_value)
        elif par == 'delay_months':
            p_value = relativedelta(rcpt_dt, tax_from)
            #print('delayed months',p_value)
            if p_value.days>0:
                bal_m = 1
            p_value = p_value.years * 12 + p_value.months + bal_m
            #print(p_value)
        elif p_valid_date==True:
            #if tab=='own':
            #    col= col.replace('_',' ').title()
            p_value= ot_df[col].to_list()[0]
            #print(p_value)
        else:
            p_value=0
        #print(col,' ',par,' ',code,' ',p_value)
        params_info.loc[params_info.index[params_info['code'] == code].tolist()[0],'P_value']=p_value
        #print(code,p_value)
    return params_info

In [28]:
def extract_parameters(regn_no,tax_from,rcpt_dt,params_info,param_info_tbo,param_info_owner,param_info_other,new_vch,pur_cd,tax_mode,final_rate):
    if len(param_info_owner)>0:
        owner_df = vt_owner.loc[vt_owner['regn_no']==regn_no]
        owner_df= copy.deepcopy(owner_df)
    if len(param_info_tbo)>0:
        last_rcpt = tbo_last_rcpt(regn_no,rcpt_dt)
        #print(last_rcpt)
        if last_rcpt !=0:
            tbo_df = vt_tax_based_on.loc[vt_tax_based_on['rcpt_no']==last_rcpt]
            c = tbo_df.columns
            #for j in c :
                #p= j.replace('_',' ').title()
                #tbo_df.rename(columns={j: p},inplace=True)
            tbo_df= copy.deepcopy(tbo_df)
        else:
            tbo_df=[]
    #print(tbo_df)
    # Extract parameters from the TBO table
    if len(tbo_df)>0 :
        #print('TBO')
        tab = 'tbo'
        #print(tab)
        params_info = owner_or_tbo(regn_no,params_info,param_info_tbo,tbo_df,tab,new_vch,rcpt_dt,tax_mode)
        #print('TBO',params_info['P_value'].to_list())
    else:
        #print('Owner')
        tab='own'
        params_info = owner_or_tbo(regn_no,params_info,param_info_owner,owner_df,tab,new_vch,rcpt_dt,tax_mode)
    #print('Shape',params_info)
    if len(param_info_other)>0:  
        #print('Other')
        #print(param_info_other)
        for pi in range(len(param_info_other)):
            col = param_info_other['column_name'].to_list()[pi][1:-2]
            par = param_info_other['column - slab_field'].to_list()[pi]
            code = param_info_other['code'].to_list()[pi]
            
            #print(null_df)
            #print('Yes!',)
            #print('Other',code,col,par)
            #if math.isnan(params_info[params_info['code']==code]['P_value'].to_list()[0]):
            if pd.isnull(params_info.loc[params_info['code']==code]['P_value']).to_list()[0]==True:
                if col == 'domain_cd':
                    p_value = vt_permit[vt_permit['Regn No']==regn_no]['Domain Cd'].to_list()
                    if len(p_value)!=0:
                        p_value = p_value[0]
                    #elif len(vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list())>0:
                    #    p_value = vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['domain_cd'].to_list()[0]
                    else:
                        p_value = None
                        #print('No Domain Cd Available')
                elif col == 'pmt_type':
                    p_value = vt_tax_based_on[vt_tax_based_on['regn_no']==regn_no]['pmt_type'].to_list()
                    if len(p_value)!=0:
                        p_value = p_value[0]
                    #elif len(vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['pmt_type'].to_list())>0:
                    #    p_value = vt_tax_based_on[vt_tax_based_on['rcpt_no']==tbo_last_rcpt(regn_no,rcpt_dt)]['pmt_type'].to_list()[0]
                    else:
                        p_value = None
                        #print('No Permit Type Available')
                elif col== 'pur_cd':
                    if pur_cd==57 or pur_cd == 58:
                        p_value=58
                elif col=='tax_amt':
                    p_value = final_rate
                elif col=="tax_period_sr_no":
                    p_value=1
                #print('Other',code,col,par)
                params_info.loc[params_info.index[params_info['code'] == code].tolist()[0],'P_value']=p_value  
    return params_info

In [83]:
date_format = "%d-%b-%Y"
def calculate_tax(regn_no,tax_from,tax_upto='2019-10-03', pur_cd = 58,final_rate=0,tax_mode='Q' ,state_cd='AS',off_cd=141,tm='n'):
    rcpt_dt = tax_upto
    #tm='Q'
    if len(vt_owner[vt_owner['regn_no']==regn_no])>0:
        vh_class = vt_owner[vt_owner['regn_no']==regn_no]['vh_class'].to_list()[0]
        new_vch = 'N'
        #print(vh_class)
    else:
        print(i,'NO INFO IN VT_OWNER',regn_no)
        return 0
        
    #slab_codes = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) & (slab_new['tax_mode']==tax_mode) & (slab_new['state_cd']==state_cd) & (slab_new['date_from']<tax_from) & (slab_new['date_to']>tax_from) ]['slab_code'].to_list()
    #tax_from = datetime.strptime(tax_from,'%d-%b-%Y')
    #print(vh_class, pur_cd, state_cd,tax_mode, tax_from)
    slab_new1 = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) & (slab_new['tax_mode']==tax_mode) &(slab_new['state_cd']==state_cd) & (slab_new['date_to'] > tax_from)]
    if len(slab_new1)==0:
        slab_new1 = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) & (slab_new['tax_mode']=='Q') &(slab_new['state_cd']==state_cd) & (slab_new['date_to'] > tax_from)]
        tm = 'q'
    #slab_new2 = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) & (slab_new['tax_mode']==tax_mode) &(slab_new['state_cd']==state_cd) & (slab_new['date_to'] > tax_upto)] # (slab_new['date_from'] < tax_from) & 
    #print(slab_new1)
    slab_codes = slab_new1['slab_code'].to_list()
    #print(pur_cd,'Slab Codes Applicable:',slab_codes)
    if len(slab_codes)==0:
        #print(i,'Missed',regn_no,'vh_class',vh_class)
        return 0

    formula_df = slab_add_new.loc[(slab_add_new['slab_code'].isin(slab_codes)) & (slab_add_new['state_cd']==state_cd)]
    formula_df = formula_df[formula_df['condition_formula']!='0=0']
    #formula_df = formula_df[formula_df['condition_formula']!='1=1']
    #formula_df.columns
    
    #print('formula_df',formula_df)
        
    #Identify the parameters required for calculating the formulas
    params = []
    for j in range(len(formula_df)):
        formula = formula_df['condition_formula'].to_list()[j] +' '+formula_df['rate_formula'].to_list()[j]
        for k in re.split(r'[()\*\s]\s*',formula):
            if '<' in k and '>' in k:
                params.append(k[k.find('<'):(k.find('>')+1)])
    params = list(set(params))
    #print('Required Parameters :',params)
    params_info = slab_fields.loc[slab_fields['code'].isin(params)]
    params_info = params_info.reindex(columns = params_info.columns.tolist() + ['P_value'])
    #print(params_info)
    param_info_owner = params_info[params_info['def_table']=='vt_owner']
    #print('Owner',param_info_owner['column - slab_field'].to_list())
    param_info_other = params_info[params_info['def_table']!='vt_owner']
    #print('Other',param_info_other['column - slab_field'].to_list())
    #param_info_tbopi = params_info[params_info['table - slab_field']=='vt_tax_based_on_permit_info']
    param_info_tbo =  params_info[params_info['table - slab_field']=='vt_tax_based_on']
    #print('TBO',param_info_tbo['column - slab_field'].to_list())
    #print('Initial Shape',params_info.shape)
    params_info = extract_parameters(regn_no,tax_from,rcpt_dt,params_info,param_info_tbo,param_info_owner,param_info_other,new_vch,pur_cd,tax_mode,final_rate)
    #print('Final Shape',params_info.shape)                                                                        
    #if pur_cd ==58:        
    #print(params_info)
    ind = calculate_formula(formula_df,params_info)[0]
    summary_df = calculate_formula(formula_df,params_info)[1]
    #if pur_cd==57:
    #print(summary_df)
    if len(ind)==0:
        print('No True Condition')
        return 0
    #Module for calculation of rates for true conditions
    true_df = formula_df.loc[ind,:]
    true_df.reset_index(drop=True, inplace=True)
    #print('True_df',true_df)

    #print(t_params)
    true_df_t = true_df[(true_df['type_flag']=='T') | (true_df['type_flag']=='M') | (true_df['type_flag']=='X') ]
    
    true_df_r = true_df[true_df['type_flag']=='R']
    true_df_s = true_df[true_df['type_flag']=='S']
    
    true_df_t = calculate_rates(true_df_t,params_info)
    #print('T',true_df_t)
    #print('S',true_df_s)
    #print(rates)
    #print('True_df',true_df_t)
    t_df = true_df_t[(true_df_t['type_flag']=='T')]
    t_ind = t_df['slab_code'].to_list()
    t_df_f_dt = slab_new1.loc[(slab_new1['slab_code'].isin(t_ind)) ,['date_from','slab_code']]
    t_df = pd.merge(t_df,t_df_f_dt,on='slab_code',how='inner')
    t_df = t_df.sort_values(by='date_from', ascending=True)
    t_df.reset_index(drop=True, inplace=True) 
    #print(t_df)
    if len(t_df)>0:
        #t_rate = true_df_t[true_df_t['type_flag']=='T']['Rate'].to_list()[0]
        #print('t_rate',t_rate)
        #if len(t_df)>1:
        t_rate = 0
        upt = datetime.strptime('01-Jan-1900','%d-%b-%Y')
        #until = max(slab_new1[slab_new1['slab_code'] in t_df['slab_code'].to_list() ]['date_to'].to_list())
        for t in range(len(t_df)):
            sc = t_df.loc[t,'slab_code']
            f_dt = slab_new1.loc[slab_new['slab_code']==sc]['date_from']
            t_dt = slab_new1.loc[slab_new['slab_code']==sc]['date_to']
            fro = max(f_dt.item(),tax_from,upt)   
            upt = min(t_dt.item(),tax_upto)
            n_days = (upt - fro).days + 1
            n_quart = math.ceil(n_days/92) +1
            if calendar.isleap(fro.year):
                days_in_year = 366
            else:
                days_in_year = 365
            if tm=='q':
                per_day_rate = t_df.loc[t,'Rate'] / 92                
            else:
                per_day_rate = t_df.loc[t,'Rate'] / days_in_year
            t_rate = t_rate + per_day_rate *n_days
            #print(pur_cd,t_rate,fro,upt, days_in_year,t_df.loc[t,'Rate'],per_day_rate,n_days,tm)
    #Module for dealing Minimum M and Amount T 
    if len(set(true_df_t['condition_formula'].to_list()))>0:
        #true_df_t = true_df_t[true_df_t['condition_formula']!='1=1']
        if len({'T','M'} - set(true_df_t['type_flag'].to_list()))==0:
            if true_df_t[true_df_t['type_flag']=='T']['Rate'].to_list()[0]> true_df_t[true_df_t['type_flag']=='M']['Rate'].to_list()[0]:
                final_rate = t_rate
            else:
                final_rate = true_df_t[true_df_t['type_flag']=='M']['Rate'].to_list()
        elif len({'T','X'} - set(true_df_t['type_flag'].to_list()))==0:
            #print('Here')
            final_rate = true_df_t[true_df_t['type_flag']=='X']['Rate'].to_list()
        else:
            final_rate = t_rate
    #else:       
    #    final_rate = true_df['Rate'].to_list()
    #else:
    #    final_rate = 0
    #print(final_rate)
    #tax_mode== 'Q'
    #Module for Surcharge
    if len(true_df_s) > 0 :
        params_info.loc[params_info['code'] == '<58>','P_value']=final_rate
        true_df_s = calculate_rates(true_df_s,params_info)
        #print(true_df_s)
        surcharge = true_df_s[true_df_s['type_flag']=='S']['Rate'].to_list()
        #print(final_rate,surcharge)
        final_rate = final_rate + surcharge[0]
    return final_rate
    '''
    if len(true_df_r) > 0:
        rr_formula = true_df_r[true_df_r['condition_formula']==f]['rate_formula'].to_list()[0]
        ra_formula = condition_form_convert(reb_params,params_info,rr_formula)
        reb_rate_result = eval(ra_formula.lower())              
        final_rate = final_rate - reb_rate_result
         
    #Module for Tax Mode Remaining years
    if tax_mode in ['Y15','Y10','Y12','Y20']:
        tax_period = relativedelta(datetime.strptime(tax_upto, date_format), datetime.strptime(tax_from, date_format))
        tax_period = tax_period.years*365 + tax_period.months*30 + tax_period.days
        #print('Tax Upto - Tax From =',tax_period,'days')
        veh_age = relativedelta(datetime.strptime(tax_from, date_format), datetime.strptime(vt_owner[vt_owner['Regn No']==regn_no]['Purchase Dt'].to_list()[0],'%Y-%m-%d'))
        #print(((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days))
        if tax_mode=='Y10':
            days_rem = 3652 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            final_rate = (final_rate/3652)*days_rem
        if tax_mode=='Y12':
            days_rem = 4383 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            #print('Days remaining in ',tax_mode,'=',days_rem,'days')
            final_rate = (final_rate/4383)*days_rem
        if tax_mode=='Y15':
            days_rem = 5478 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
            final_rate = (final_rate/5478)*days_rem
    '''                            
    #Module for Quarterly Mode
    '''
    if tax_mode== 'Q':
        #print(tax_upto,' ',tax_from)
        tax_period = relativedelta(tax_upto, tax_from)
        #print(tax_period)
        years_q = tax_period.years * 4 
        #tax_period = tax_period.years*12 + tax_period.months
        if tax_period.months <=3:
            months_q = 1
        elif tax_period.months <=6:
            months_q = 2
        elif tax_period.months <=9:
            months_q = 3
        else:
            months_q = 4
        n_quart = years_q + months_q
        #print('n_quart',n_quart,final_rate,final_rate*n_quart)
        final_rate = final_rate * n_quart
    elif tax_mode == 'Y':
        tax_period = relativedelta(tax_upto, tax_from)
        years_y = tax_period.years * 12 
        months_y = math.ceil(tax_period.months + years_y)
        if tax_period.days > 0:
            months_y = months_y+1
        #print((final_rate / 12),months_y)
        if tm == 'y2q':
            months_to_quart = math.ceil(months_y/3)
            final_rate = (final_rate / 4) * months_to_quart
        else:
            final_rate = (final_rate / 12) * months_y
    '''
    

In [84]:
nc_regn_no_list = []
for i in range(0,10):   
    regn_no = def_list['Regn No'].to_list()[i]    
    state_cd = state
    #print(regn_no,state)
    #off_cd = 1
    vh_class = def_list['Vehicle Class'].to_list()[i]
    #rcpt_no = def_list['Rcpt No'].to_list()[i]
    #print(regn_no,rcpt_no)
    #tax_amt = def_list['Tax Amt'].to_list()[i]
    #rcpt_dt = def_list['Rcpt Dt'].to_list()[i]
    
    #tax_from = def_list['latest_tax_paid_upto'] + 1
    tax_from = def_list['Tax Arrear From'].to_list()[i]
    #print(tax_from)
    tax_from = datetime.strptime(tax_from, date_format)
    
    tax_upto = def_list['Tax Arrear UpTo'].to_list()[i]
    tax_upto = datetime.strptime(tax_upto,'%d-%b-%Y')
    #tax_upto = datetime.strptime('04-May-2020','%d-%b-%Y')
    #print('Delayed',tax_upto-tax_from)
    c_tax_amt = round(calculate_tax(regn_no,tax_from,tax_upto,tax_mode='Y',state_cd=state))
    #print(c_tax_amt)
    if c_tax_amt == 0:
        #print('Unable to calculate tax')
        nc_regn_no_list.append(regn_no)
    tax_amt = pd.to_numeric(def_list['Tax Amount in (Rs)'].str.replace(',',''),errors='coerce').to_list()[i]
    
    tax_fine = pd.to_numeric(def_list['Tax Fine in (Rs)'].str.replace(',',''),errors='coerce').to_list()[i]
    if c_tax_amt !=0:
        c_tax_fine = round(calculate_tax(regn_no,tax_from,tax_upto, pur_cd = 57, final_rate = c_tax_amt,tax_mode='Y',state_cd=state))
    #print(tax_fine)
    if tax_amt!=c_tax_amt: 
        print('Mismatch',i,regn_no, vh_class,str(tax_from)[:11],str(tax_upto)[:11],tax_amt,c_tax_amt,c_tax_fine,tax_fine)
    else:
        print(i, regn_no,vh_class,str(tax_from)[:11],str(tax_upto)[:11], tax_amt, c_tax_amt,c_tax_fine,tax_fine)
    #summary_df = c_tax_amt[1]
    

Mismatch 0 ADI8318 Goods Carrier 2011-04-01  2021-03-31  54078 53388.0 267159.0 23439.0
Mismatch 1 AME5602 Goods Carrier 2016-04-01  2021-03-31  28440 27676.0 69039.0 10620.0
Mismatch 2 AMS2949 Goods Carrier 2014-11-01  2020-10-31  33378 32665.0 98084.0 13089.0
Mismatch 3 AP09V9947 Goods Carrier 2010-07-01  2020-06-30  52728 52037.0 260399.0 22752.0
Mismatch 4 AP09V9950 Goods Carrier 2010-07-01  2020-06-30  52728 52037.0 260399.0 22752.0
Mismatch 5 AP09W3109 Goods Carrier 2010-07-01  2020-06-30  52728 52037.0 260399.0 22752.0
Mismatch 6 AP09W3113 Goods Carrier 2010-07-01  2020-06-30  52728 52037.0 260399.0 22752.0
Mismatch 7 AP09W3114 Goods Carrier 2010-07-01  2020-06-30  52728 52037.0 260399.0 22752.0
Mismatch 8 AP09W3117 Goods Carrier 2010-07-01  2020-06-30  52728 52037.0 260399.0 22752.0
Mismatch 9 AP26X8611 Goods Carrier 2012-08-01  2020-07-31  75792 74088.0 295745.0 31056.0


In [30]:
def_list = pd.read_excel(r'C:\Users\CDMA\Downloads\Tripura defaulters all.xls')
#def_list = def_list[def_list['Tax Arrear From'].notnull()]
#def_list.columns

In [31]:
def_list.shape

(38536, 30)

In [32]:
def_list = def_list[def_list['Tax Amount in (Rs)'].notnull()]
#def_list = def_list[def_list['Vehicle Class']=='Motor Cab']
def_list = def_list[def_list['Tax Arrear From'].notnull()]
def_list = def_list[def_list['Tax Amount in (Rs)'].notnull()]
def_list = def_list[def_list['Tax Type']=='MV Tax']
def_list.reset_index(drop=True, inplace=True)

In [33]:
def_list['Vehicle Class'].value_counts()

Three Wheeler (Passenger)    9496
Goods Carrier                8159
M-Cycle/Scooter              3663
Motor Cab                    2377
Three Wheeler (Goods)        1946
Motor Car                    1496
Bus                           528
Crane Mounted Vehicle         208
Excavator (Commercial)        194
Tractor (Commercial)          131
Earth Moving Equipment        114
Maxi Cab                       91
Trailer (Commercial)           70
Moped                          67
Excavator (NT)                 57
Omni Bus                       30
Agricultural Tractor           27
Ambulance                      14
Camper Van / Trailer            3
e-Rickshaw(P)                   1
Fire Fighting Vehicle           1
Power Tiller (Commercial)       1
Name: Vehicle Class, dtype: int64

In [34]:
def_list = def_list[def_list['Vehicle Class']=='Goods Carrier']

In [46]:
vt_tax_based_on[vt_tax_based_on['regn_no']=='LT07003722']['other_criteria']

Series([], Name: other_criteria, dtype: int64)

In [27]:
vt_tax_based_on.columns

Index(['Number of Records', 'ac_fitted', 'audio_fitted', 'cubic_cap',
       'distance_run_in_quarter', 'domain_cd', 'fin_yr_sale_amt', 'floor_area',
       'fuel', 'gcw', 'height', 'hp', 'imported_vch', 'ld_wt', 'length',
       'no_adv_units', 'no_cyl', 'no_of_trips', 'off_cd', 'op_dt',
       'other_criteria', 'pmt_catg', 'pmt_type', 'purchase_dt', 'rcpt_no',
       'regn_no', 'regn_type', 'route_class', 'route_length', 'sale_amt',
       'seat_cap', 'service_type', 'sleeper_cap', 'stand_cap', 'state_cd',
       'unld_wt', 'vch_catg', 'vch_purchase_as', 'vh_class', 'video_fitted',
       'wheelbase', 'width'],
      dtype='object')

In [47]:
vt_owner[vt_owner['regn_no']=='LT07003722']

,Number of Records,ac_fitted,annual_income,audio_fitted,body_type,c_add1,c_add2,c_add3,c_district,c_pincode,...,stand_cap,state_cd,status,unld_wt,vch_catg,vch_purchase_as,vh_class,video_fitted,wheelbase,width
1631792,1,N,NaN,N,NaN,NaN,NaN,NaN,0.0,0,...,0.0,MZ,A,9420,HGV,NaN,59,N,0.0,0.0


In [73]:
t_ind = [4,22,56]
x = slab_new.loc[(slab_new['slab_code'].isin(t_ind)) & (slab_new['state_cd']=='TR') & (slab_new['vh_class']==59) & (slab_new['tax_mode']=='Y') & (slab_new['pur_cd']==58),['date_from','slab_code']]
y = slab_new[(slab_new['vh_class']==59) & (slab_new['pur_cd']==58) & (slab_new['tax_mode']=='Y') &(slab_new['state_cd']=='TR') & (slab_new['date_to'] > tax_from)]
t_df = pd.merge(t_df,t_df_f_dt,on='slab_code',how='inner')
t_df.sort_values(by='date_from', ascending=True)

,date_from_x,slab_code,class_type,date_from_y,date_to,descr,emp_cd,op_dt,pur_cd,state_cd,tax_mode,vh_class
0,2000-01-01,22,1,2000-01-01,2011-10-31,GOODS CARRIER TAX OLD,200016,03-11-2015 11:39,58,TR,Y,59.0
2,2011-01-11,4,1,2011-01-11,2018-09-07,HEAVY GOODS VEHICLE,200116,09-07-2018 13:59,58,TR,Y,59.0
1,2018-10-07,56,1,2018-10-07,2099-03-31,GOODS VEHICLE TAX 2018,200116,09-11-2018 10:21,58,TR,Y,59.0
